In [11]:


import pandas as pd
import numpy as np
from pandas import DataFrame
import os
os.chdir('G:\\Banglalink\\Python Master\\PythonForModellers-master\\Data')

In [4]:
data=pd.read_csv('terror.csv',sep=',',header=0, encoding="latin")

print (data.head())
print (data.shape)
print (data.columns.tolist())


C:\Users\Neha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,61,62,66,116,117,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970       0     0        NaN         0        NaN       58   
1  197000000002   1970       0     0        NaN         0        NaN      130   
2  197001000001   1970       1     0        NaN         0        NaN      160   
3  197001000002   1970       1     0        NaN         0        NaN       78   
4  197001000003   1970       1     0        NaN         0        NaN      101   

          country_txt  region   ...    addnotes scite1 scite2  scite3  \
0  Dominican Republic       2   ...         NaN    NaN    NaN     NaN   
1              Mexico       1   ...         NaN    NaN    NaN     NaN   
2         Philippines       5   ...         NaN    NaN    NaN     NaN   
3              Greece       8   ...         NaN    NaN    NaN     NaN   
4               Japan       4   ...         NaN    NaN    NaN     NaN   

   dbsource  INT_LOG  INT_IDEO INT_MISC INT_ANY  related  
0      PGIS    

In [5]:

### How many attacks happened in India
data['country_txt'].unique().tolist()
data[data['country_txt']=="India"]
data[data['country_txt']=="India"].shape[0]


9940

In [6]:
## How many attacks happened in India and upto 3 people were killed
data[(data['country_txt']=='India')&(data['nkill']<=3)].shape[0]


8362

In [7]:
# This looks ugly
data.query("country_txt=='India' & nkill<=3").shape[0]


8362

In [ ]:
# Extract the city and summary for attacks above
data.query("country_txt=='India' & nkill<=3")[['city','summary']].head(10)


In [ ]:
# In a single terror incident in India, find out top 5 cities by number killed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)['city'].head(5)


In [ ]:
# Little more detailed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)[['city','nkill','iyear']].head(5)


In [ ]:

# In a single terror incident in India, find out top 5 cities by number killed and wounded
data.query("country_txt=='India'").sort_values(['nkill','nwound'],ascending=[False,False])['city'].head(5)


In [ ]:
#### Adding new columns to the data
#Attacks which were successful and were suicide attacks
data['success_suicide']=data['suicide']+data['success']
data=data.assign(success_suicide_1=data['suicide']+data['success'])

data.query("success_suicide==2").shape[0]


In [ ]:

### Dropping columns
data.drop('success_suicide',axis=1) 
data.drop('success_suicide',axis=1,inplace=True) 


In [ ]:

### Aggregations and manipulations using apply and map
#map: map a function to each element of a series object
data['nkill'].describe()
#Suppose we want to label all the incidents where the number killed was more than 5 as severe. This would involve applying a function on each element of the series, map helps in doing that
def get_label(x):
    if x>5:
        return 'Severe'
    else:
        return 'Not Severe'
data['nkill'].map(get_label)
#You can use lambda functions as well (if else follows a special form when used with lambdas )
data['nkill'].map(lambda x: "Severe" if x>5 else "Not Severe")

In [ ]:
# We can use apply to use a function row wise or column wise
# Let's write a function to label an incident that was both successful and suicidal

def get_label(row):
    if row['success']==1 and row['suicide']==1:
        return 1
    else:
        return 0
data.apply(get_label,axis=1)
data.apply(get_label,axis=1).unique()

data.apply(lambda row: 1 if row['success']==1 and row['suicide']==1 else 0,axis=1)


In [12]:

### Create a new  category representing if the incident occured in Afghanistan, Pakistan or India as one level of the category and all the other countries as another level

def get_label(row):
    if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan':
        return 'Af-Pak-India'
    else:
        return 'ROW'
data.apply(get_label,axis=1)

data['Local']=data.apply(get_label,axis=1)


In [15]:
data.apply(lambda row: 'Af-Pak-India' if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan' else  'ROW', axis =1 )

0                  ROW
1                  ROW
2                  ROW
3                  ROW
4                  ROW
5                  ROW
6                  ROW
7                  ROW
8                  ROW
9                  ROW
10                 ROW
11                 ROW
12                 ROW
13                 ROW
14                 ROW
15                 ROW
16                 ROW
17                 ROW
18                 ROW
19                 ROW
20                 ROW
21                 ROW
22                 ROW
23                 ROW
24                 ROW
25                 ROW
26                 ROW
27                 ROW
28                 ROW
29                 ROW
              ...     
156742             ROW
156743             ROW
156744             ROW
156745             ROW
156746             ROW
156747             ROW
156748             ROW
156749             ROW
156750             ROW
156751             ROW
156752             ROW
156753             ROW
156754     

In [16]:

## Number of incidents in Af-Pak-India vs ROW
data['Local'].value_counts()
data.groupby('Local').size()

Local
Af-Pak-India     22708
ROW             134064
dtype: int64

In [17]:


## Number of suicide attacks by Af-Pak-India vs ROW
data.groupby(['Local','suicide']).agg({'eventid':np.size})
data.groupby(['Local','suicide'],as_index=False).agg({'eventid':np.size})


,Local,suicide,eventid
0,Af-Pak-India,0,22228
1,Af-Pak-India,1,480
2,ROW,0,129773
3,ROW,1,4291


In [18]:

## Number of suicides attacks and average kills by Af-Pak-India vs ROW
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size])
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index()

,Local,suicide,mean,size
0,Af-Pak-India,0,1.539059,22228.0
1,Af-Pak-India,1,13.157620,480.0
2,ROW,0,2.192427,129773.0
3,ROW,1,10.159229,4291.0


In [19]:
#Rename columns
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index().rename(columns={'mean':'Average_Kills','size':'Number_Incidents'})

data.groupby(["Local",'suicide']).agg({'nkill':np.mean,'nwound':np.sum}).reset_index().rename(columns={'nwound':'Total_Wounded','nkill':'Number_Killed'})


,Local,suicide,Number_Killed,Total_Wounded
0,Af-Pak-India,0,1.539059,53341.00
1,Af-Pak-India,1,13.157620,11928.00
2,ROW,0,2.192427,292321.31
3,ROW,1,10.159229,82946.69
